# Replacing User Latent Vector With Behavioural Sequence RNN Output For CF Recommendation

##### For the situations that we * don't have fresh user embedding* 

#### Good side of user embedding
The latent space empowers the model to featurize the characters of user, entirely by learning.
#### Bad side of user embedding
Major problems of user embedding is always around the new user:

* We train a model with [latent cf + neural network](3.1.3_recommender_system.ipynb)
* Usually we won't retrain a model within the same day.
* If the new user some with his/her preference record, we have to train it in a way within the consistency of the old user's latent vector
* So we can't really apply the adavanced model to the fresh users.

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# %cd /data
# !!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
# !!unzip ml-latest-small.zip

In [3]:
%ls /data/ml-latest-small/

README.txt   links.csv    movies.csv   ratings.csv  tags.csv


In [42]:
DATA = "/data/ml-latest-small/"
DIM = 100 # dimension for embedding
SEQ_LEN = 19

In [5]:
files = os.listdir(DATA)
files

['links.csv', 'tags.csv', 'ratings.csv', 'README.txt', 'movies.csv']

In [6]:
data = dict()
for f in files:
    if f[-3:]=="csv":
        data[f.split(".")[0]] = pd.read_csv(DATA+f)

### Check the data

In [7]:
from IPython.display import display
list(display(k,v.sample(5)) for k,v in data.items())

'links'

,movieId,imdbId,tmdbId
6733,53996,418279,1858.0
3371,4232,227538,10054.0
2038,2542,120735,100.0
2959,3705,99141,1727.0
7732,84187,860906,22843.0


'tags'

,userId,movieId,tag,timestamp
915,547,2936,afi,1182393939
434,364,6539,sword fight,1444529953
864,546,48780,based on a book,1301715225
417,364,4973,quirky,1444528852
1193,547,112070,tivo,1476113970


'ratings'

,userId,movieId,rating,timestamp
61910,452,2112,3.0,976420133
35925,260,4995,2.5,1207985091
52106,380,89470,4.0,1344290433
94093,624,3596,1.0,1044272089
4743,24,380,4.0,849282414


'movies'

,movieId,title,genres
5681,25841,Stage Door (1937),Drama
1243,1562,Batman & Robin (1997),Action|Adventure|Fantasy|Thriller
3479,4397,Cannonball Run II (1984),Action|Comedy
3901,5038,"Flight of Dragons, The (1982)",Adventure|Animation|Children|Drama|Fantasy
2532,3148,"Cider House Rules, The (1999)",Drama


[None, None, None, None]

In [8]:
rate_df = data["ratings"]
len(rate_df)

100004

In [9]:
userId = list(set(data["ratings"]["userId"]))
movieId = list(set(data["ratings"]["movieId"]))
print(len(userId),len(movieId))

671 9066


In [10]:
u2i = dict((v,k) for k,v in enumerate(userId))
m2i = dict((v,k) for k,v in enumerate(movieId))
i2u = dict((k,v) for k,v in enumerate(userId))
i2m = dict((k,v) for k,v in enumerate(movieId))

In [11]:
rate_df["movieIdx"] = rate_df.movieId.apply(lambda x:m2i[x]).astype(int)
rate_df["userIdx"] = rate_df.userId.apply(lambda x:u2i[x]).astype(int)
rate_df["rating"] = rate_df["rating"]/5

In [12]:
gb = rate_df.sort_values(by=["userIdx","timestamp"]).groupby("userIdx")

gb.apply(lambda x:x.sample(n = 20, replace = False))

userId  movieId  rating   timestamp  movieIdx  userIdx
userIdx                                                              
0       6           1     1287     0.4  1260759187      1149        0
        11          1     1405     0.2  1260759203      1260        0
        10          1     1371     0.5  1260759135      1226        0
        18          1     2968     0.2  1260759200      2616        0
        3           1     1129     0.4  1260759185      1004        0
        0           1       31     0.5  1260759144        30        0
        12          1     1953     0.8  1260759191      1697        0
        2           1     1061     0.6  1260759182       949        0
        8           1     1339     0.7  1260759125      1195        0
        17          1     2455     0.5  1260759113      2169        0
        5           1     1263     0.4  1260759151      1126        0
        15          1     2193     0.4  1260759198      1932        0
        7           1     1293     0.4  1260759148      1155        0
        4           1     1172     0.8  1260759205      1038        0
        9           1     1343     0.4  1260759131      1199        0
        19          1     3671     0.6  1260759117      3207        0
        14          1     2150     0.6  1260759194      1894        0
        1           1     1029     0.6  1260759179       921        0
        13          1     2105     0.8  1260759139      1849        0
        16          1     2294     0.4  1260759108      2015        0
1       29          2      150     1.0   835355395       142        1
        33          2      168     0.6   835355710       160        1
        84          2      552     0.6   835355860       530        1
        65          2      382     0.6   835356165       369        1
        73          2      485     0.6   835355918       463        1
        44          2      265     1.0   835355697       253        1
        43          2      261     0.8   835355681       249        1
        90          2      590     1.0   835355395       560        1
        91          2      592     1.0   835355395       562        1
        88          2      588     0.6   835355441       558        1
...               ...      ...     ...         ...       ...      ...
669     99865     670      110     0.6   938782093       108      669
        99864     670       50     1.0   938781934        49      669
        99859     670       25     1.0   938782344        24      669
        99871     670      593     1.0   938782234       563      669
        99884     670     2723     0.4   940944033      2404      669
        99862     670       36     0.8   938782050        35      669
        99878     670     1923     0.4   938782414      1667      669
        99876     670     1617     0.4   938781934      1433      669
        99883     670     2683     1.0   938782281      2367      669
        99873     670     1183     1.0   938782193      1048      669
670     99951     671     2858     0.8  1063503841      2521      670
        99930     671     1673     0.7  1063500961      1479      670
        99911     671     1090     0.8  1064891507       977      670
        99912     671     1136     1.0  1064891032      1011      670
        99955     671     3052     0.2  1063503966      2692      670
        99970     671     4019     0.7  1065111959      3527      670
        99923     671     1259     0.8  1064890570      1121      670
        99992     671     5816     0.8  1065111963      4826      670
        99968     671     3996     0.7  1064245511      3505      670
        99973     671     4033     0.8  1065111954      3540      670
        99946     671     2683     0.8  1063500751      2367      670
        99891     671       50     0.9  1064890944        49      670
        99938     671     2291     1.0  1063500850      2012      670
        99986     671     5299     0.6  1065112004      4487      670
        99956     671    

In [13]:
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader

In [14]:
pick_k = np.array([0]*19 +[1])==1

In [23]:
def sample_split(x):
    sampled = x.sample(n = 20, replace = False)
    seq = sampled.head(19).sort_values(by="timestamp")
    y = sampled[pick_k]
    return pd.concat([seq,y])

class rnn_record(Dataset):
    def __init__(self, gb):
        self.gb = gb
        self.make_seq()
    
    def make_seq(self):
        self.all_seq = self.gb.apply(sample_split)
        
    def __len__(self):
        return len(self.gb)
        
    def __getitem__(self,idx):
        df = self.all_seq.loc[idx]
        seq = df.head(19)[["movieIdx","rating"]].values
        targ = df[pick_k][["movieIdx","rating"]].values
        targ_v, targ_y =targ[:,0], targ[:,1]
        return idx,seq,targ_v,targ_y
ds = rnn_record(gb)

In [16]:
# dl = DataLoader(ds,batch_size=32,shuffle=True)
# gen = iter(dl)

# idx,seq,targ_v,targ_y = next(gen)

# idx.size(),seq.size(),targ_v.size(),targ_y.size()

### Model

In [17]:
import torch
from torch import nn
from torch.nn import functional as F

In [36]:
class mLinkNet(nn.Module):
    def __init__(self, hidden_size,v_size):
        super(mLinkNet,self).__init__()
        self.hidden_size = hidden_size
        self.v_size = v_size
        self.emb = nn.Embedding(v_size,hidden_size)
        
        self.rnn = nn.GRU(input_size = self.hidden_size+1,
                          hidden_size= hidden_size+1,
                          num_layers=1,
                          batch_first = True,
                          dropout=0)
        
        self.mlp = nn.Sequential(*[
            nn.Dropout(.3),
            nn.Linear(hidden_size*2 + 1, 256, bias=False),
            nn.BatchNorm1d(256),
            nn.Linear(256,1,bias=False),
            nn.Sigmoid(),
        ])
    
    def forward(self,seq,targ_v):
        seq_vec = torch.cat([self.emb(seq[:,0].long()),
                             seq[:,1].unsqueeze(-1).float()], dim=2)
        output, hn = self.rnn(seq_vec)
#         print(hn.size(),self.emb(targ_v.long()).size())
        x = torch.cat([hn.squeeze(0),self.emb(targ_v.long()).squeeze(1)],dim=1)
        return self.mlp(x)

In [37]:
mln = mLinkNet(hidden_size = DIM, v_size = len(movieId))

# mln(seq, targ_v)

In [38]:
from torch.optim import Adam
from ray.matchbox import Trainer
opt = Adam(mln.parameters())
loss_func = nn.MSELoss()

In [39]:
trainer = Trainer(ds, batch_size=16, print_on=3)

In [40]:
train_len = len(trainer.train_data)
def action(*args,**kwargs):
    idx,seq,targ_v,y = args[0]
    y = y.float()
    
    opt.zero_grad()
    
    y_ = mln(seq, targ_v)
    loss = loss_func(y_,y)
    
    loss.backward()
    opt.step()
    mae = torch.mean(torch.abs(y_-y))
    if kwargs["ite"] == train_len - 1:
        trainer.train_data.dataset.make_seq()
    return {"loss":loss.item(),"mae":mae.item()}
trainer.action  = action

In [31]:
trainer.train(50)

⭐[ep_49_i_41]	loss	0.028✨	mae	0.132: 100%|██████████| 42/42 [00:02<00:00, 14.13it/s]


In [41]:
trainer.train(50)

⭐[ep_49_i_41]	loss	0.065✨	mae	0.200: 100%|██████████| 42/42 [00:02<00:00, 14.03it/s]
